In [5]:
pip show azureml-sdk

Name: azureml-sdk
Version: 1.59.0
Summary: Used to build and run machine learning workflows upon the     Azure Machine Learning service.
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: 
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/automl-env/lib/python3.9/site-packages
Requires: azureml-core, azureml-dataset-runtime, azureml-pipeline, azureml-train-automl-client, azureml-train-core
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [29]:
#pip install azureml-sdk==1.59.0

Note: you may need to restart the kernel to use updated packages.


In [30]:
#pip install azureml-sdk[automl]

  Using cached azureml_train_automl-1.59.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached azureml_automl_runtime-1.59.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached azureml_train_automl_runtime-1.59.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached azureml_responsibleai-1.59.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached azureml_training_tabular-1.59.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached dill-0.3.9-py3-none-any.whl.metadata (10 kB)
  Using cached gensim-4.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.5 kB)
  Using cached smart_open-6.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached lightgbm-3.2.1-py3-none-manylinux1_x86_64.whl.metadata (14 kB)
  Using cached pandas-1.5.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached psutil-5.9.3-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (21 kB)
  Using cached scipy-1.10.1-cp39-cp39-man

In [6]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-280179
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-280179


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "Udacity"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster")
except ComputeTargetException:
    print("Creating new cluster")

    compute_config = AmlCompute.provisioning_configuration(
        vm_size = 'STANDARD_D2_V2',
        min_nodes=0,
        max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cluster.wait_for_completion(show_output=True)

Found existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
pip install 'azureml-sdk[notebooks]'

Note: you may need to restart the kernel to use updated packages.


In [9]:
#from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.1,0.5,1,5),
        '--max_iter': choice(50, 100, 500)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./',
                script='train_wv.py',
                compute_target='Udacity',
                environment=sklearn_env)

#primary_metric = 'Accuracy'
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                hyperparameter_sampling=ps,
                policy=policy,
                primary_metric_name='Accuracy',
                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                max_total_runs=100,
                max_concurrent_runs=4)

In [16]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from collections.abc import Mapping
from azureml.widgets import RunDetails


hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_177d5a1a-721b-461f-966b-64171c3dd5f7
Web View: https://ml.azure.com/runs/HD_177d5a1a-721b-461f-966b-64171c3dd5f7?wsid=/subscriptions/9e65f93e-bdd8-437b-b1e8-0647cd6098f7/resourcegroups/aml-quickstarts-280179/workspaces/quick-starts-ws-280179&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-05-07T13:16:21.7596652Z][GENERATOR][WARNING]Space size : 12 is less than max total jobs : 100, only 12 jobs will be generated 
[2025-05-07T13:16:23.4920915Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-05-07T13:16:24.0979647Z][SCHEDULER][INFO]Scheduling job, id='HD_177d5a1a-721b-461f-966b-64171c3dd5f7_2' 
[2025-05-07T13:16:24.0966298Z][SCHEDULER][INFO]Scheduling job, id='HD_177d5a1a-721b-461f-966b-64171c3dd5f7_1' 
[2025-05-07T13:16:24.0240357Z][SCHEDULER][INFO]Scheduling job, id='HD_177d5a1a-721b-461f-966b-64171c3dd5f7_0' 
[2025-05-07T13:16:24.2282220Z][SCHEDULER][INFO]Scheduling job, id='HD_177d5a1a-721b-461f-966b-64171c3dd5f7_3' 
[2025-05

{'runId': 'HD_177d5a1a-721b-461f-966b-64171c3dd5f7',
 'target': 'Udacity',
 'status': 'Completed',
 'startTimeUtc': '2025-05-07T13:16:21.415391Z',
 'endTimeUtc': '2025-05-07T13:33:30.85731Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '6f1aa8d4-f85b-4271-96d1-aa1ccdaa56e4',
  'user_agent': 'python/3.9.21 (Linux-6.8.0-1026-azure-x86_64-with-glibc2.35) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'best_child_run_id': 'HD_177d5a1a-721b-461f-966b-64171c3dd5f7_11',
  'score': '0.9124787569798496',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_177d5a1a-721b-461f-966b-64171c3dd5f7_11'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-train'

In [17]:
#from collections.abc import Mapping
#from azureml.widgets import RunDetails

#RunDetails(hyperdrive_run).show()

In [19]:
#import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
best_run.register_model(model_name = 'best_hd_model.pkl', model_path='outputs/model.joblib')

Run(Experiment: udacity-project,
Id: HD_177d5a1a-721b-461f-966b-64171c3dd5f7_11,
Type: azureml.scriptrun,
Status: Completed)


Model(workspace=Workspace.create(name='quick-starts-ws-280179', subscription_id='9e65f93e-bdd8-437b-b1e8-0647cd6098f7', resource_group='aml-quickstarts-280179'), name=best_hd_model.pkl, id=best_hd_model.pkl:1, version=1, tags={}, properties={})

In [20]:
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '0.5', '--max_iter', '500']


In [21]:
print(best_run.get_metrics())

{'Max iterations:': 500, 'Regularization Strength:': 0.5, 'Accuracy': 0.9124787569798495}


In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [11]:
import sys
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install numpy

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... - error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      


In [12]:
from train_wv import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(data)

data_joined = pd.concat([x, y], axis=1)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [13]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_joined,
    label_column_name='y',
    n_cross_validations=2)

In [ ]:
# Submit your automl run

#experiment = Experiment(ws, 'automl')
#run = experiment.submit(config=automl_config, show_out=True)

# Submit your automl run
from azureml.core.experiment import Experiment

experiment = Experiment(ws, 'project1automl')
automl_run = experiment.submit(config=automl_config, show_out=True)

In [ ]:
# Retrieve and save your best automl model.
from azureml.core.experiment import Experiment

best_automl_run, fitted_automl_model = run.get_output()
best_automl_run.register_model(model_name = 'automl_best_run', model_path='outputs/')
print(best_automl_run)

In [ ]:
print(best_auto_run.get_details())

In [ ]:
print(best_auto_run.get_metrics())

In [ ]:
cluster.delete()